In [58]:
import os
import re
import base64
from typing import List
import pandas as pd
from dotenv import find_dotenv, load_dotenv

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever
from langchain.chat_models import init_chat_model
from langchain_core.documents import Document
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOCIGenAI
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain.tools import tool

from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    EmbedTextDetails,
    OnDemandServingMode,
)
import oracledb

_ = load_dotenv(find_dotenv())
oracledb.init_oracle_client()

UN = os.getenv("UN")
PW = os.getenv("PW")
DSN = os.getenv("DSN")
OCI_COMPARTMENT_ID = os.getenv("OCI_COMPARTMENT_ID")

### Utils

In [59]:
model = ChatOCIGenAI(
    model_id="cohere.command-a-03-2025",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=OCI_COMPARTMENT_ID,
    model_kwargs={"temperature": 1.0, "max_tokens": 500},
    )

In [60]:
def get_embedding(text: str) -> list:
  embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-multilingual-image-v3.0",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=OCI_COMPARTMENT_ID,
  )
  return embeddings.embed_query(text)

### Retriever

In [61]:
class CustomEmbeddingRetriever(BaseRetriever):
    """
    Custom image retriever.
    """

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        docs: List[Document] = []
        embed_query = str(get_embedding(query))
        try:
            with oracledb.connect(user=UN, password=PW, dsn=DSN) as connection:
                with connection.cursor() as cursor:
                    cursor.setinputsizes(oracledb.DB_TYPE_VECTOR)
                    select_sql = f"""
                        SELECT
                            id,
                            name,
                            content_type,
                            file_path,
                            summary
                        FROM
                            embedding_contents
                        ORDER BY VECTOR_DISTANCE(embedding, to_vector(:1, 1024, FLOAT32), COSINE)
                        FETCH FIRST 3 ROWS ONLY
                    """
                    cursor.execute(select_sql, [embed_query])
                    index = 1
                    for row in cursor:
                        doc = Document(
                            page_content=row[3],
                            metadata={
                                'id':row[0],
                                'name': row[1],
                                'content_type': row[2],
                                'file_path': row[3],
                                'vector_index': index
                                }
                            )
                        docs.append(doc)
                        index += 1
                connection.close()
                        
        except oracledb.DatabaseError as e:
            print(f"Database error: {e}")
            raise
        except Exception as e:
            print("Error Vector Search:", e)
        
        return docs

### Tools
* Get Contents
* Get Text with Markdown
* Get Text with Image

In [62]:
@tool
def get_content(query: str) -> str:
    """
    Get content from the database based on the query.
    Args:
        query (str): The query to search for.
    Returns:
        str: The content retrieved from the database.
    """
    retriever = CustomEmbeddingRetriever()
    result_content = retriever.invoke(query)
    result = result_content[0]
    print(f"result_content: {result}")
    
    if result.metadata['content_type'] == 'image':
        print(f"image_content: {result.metadata['name']}")
        return {"content": result, "type": "image"}
    elif result.metadata['content_type'] == 'text':
        print(f"text_content: {result.metadata['name']}")
        return {"content": result, "type": "text"}
    else:
        return {"error": "Unsupported content type"}


In [63]:
@tool
def get_text_with_markdown(query: str, content: Document) -> str:
    """
    Get the value of department expenses, sales, and operating income by text with markdown retriever
    Args:
        query (str): The query to ask the model. YYYY/MM format is required.
        content (Document): The document containing the content to be used as context.
    Returns:
        str: The answer from the model.
    """
    
    # retriever = CustomEmbeddingRetriever()
    # result_content = retriever.invoke(query)
    id = content.metadata['id']

    docs: List[Document] = []
    df = pd.DataFrame()
    try:
        with oracledb.connect(user=UN, password=PW, dsn=DSN) as connection:
            with connection.cursor() as cursor:
                cursor.setinputsizes(oracledb.DB_TYPE_VECTOR)
                select_sql = f"""
                    SELECT
                        id,
                        markdown
                    FROM
                        embedding_contents
                    WHERE id = :1
                """
                cursor.execute(select_sql, [id])
                for row in cursor:
                    df_tmp = pd.DataFrame([[row[0], row[1].read()]],
                                            columns=["id", "markdown"])
                    df = pd.concat([df, df_tmp], ignore_index=True)
                
                for i in range(len(df)):
                    id = df.iloc[i, 0]
                    markdown = df.iloc[i, 1]
                    doc = Document(
                        page_content=markdown,
                        metadata={'id':id, 'vector_index': i}
                        )
                    docs.append(doc)
            connection.close()
                    
    except oracledb.DatabaseError as e:
        print(f"Database error: {e}")
        raise
    except Exception as e:
        print("Error Vector Search:", e)
    
    llm = ChatOCIGenAI(
        model_id="cohere.command-a-03-2025",
        service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
        compartment_id=OCI_COMPARTMENT_ID,
        )
    
    prompt = ChatPromptTemplate([
        ("system", "あなたは質疑応答のAIアシスタントです。必ず日本語で答えてください。"),
        ("human", """
         以下のMarkdownのコンテキストに基づいて質問に答えてください。
         回答は数字だけを回答してください。
         ** 質問 **
          {query} 
          
        ** コンテキスト **
        """
        f"{docs[0].page_content}"
        ),
    ])

    chain = {'query': RunnablePassthrough()} | prompt | llm | StrOutputParser()

    result = chain.invoke(query)
    return result

In [64]:
def get_text_by_image(query: str):
    """
    Text to Image
    """

    retriever = CustomEmbeddingRetriever()
    result_images = retriever.invoke(query)
    
    id = result_images[0].metadata['id']
    file_name = os.path.basename(result_images[0].metadata['file_path'])
    print(f"file_name: {file_name}")
    try:
        with oracledb.connect(user=UN, password=PW, dsn=DSN) as connection:
            with connection.cursor() as cursor:
                cursor.setinputsizes(oracledb.DB_TYPE_VECTOR)
                select_sql = f"""
                    SELECT
                        image_blob
                    FROM
                        embedding_contents
                    WHERE id = :1
                """
                cursor.execute(select_sql, [id])
                blob, = cursor.fetchone()
                offset = 1
                bufsize = 65536

                with open("../images/" + file_name, 'wb') as f:
                    while True:
                        data = blob.read(offset, bufsize)
                        if data:
                            f.write(data)
                        if len(data) < bufsize:
                            break
                        offset += bufsize
            connection.close()
                    
    except oracledb.DatabaseError as e:
        print(f"Database error: {e}")
        raise
    except Exception as e:
        print("Error Vector Search:", e)
    
    return result_images[0]

In [65]:
def chat_with_image(image_path: str, prompt: str, system_prompt: str = None) -> str:
  with open(image_path, "rb") as img_file:
    image_data = base64.b64encode(img_file.read()).decode("utf-8")
  
  prompt_with_image = [
    SystemMessage(
        content=system_prompt
    ),
    HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": "data:image/png;base64,"+image_data,
            }
        },
        ]
      )
  ]

  llm = ChatOCIGenAI(
      model_id="meta.llama-4-scout-17b-16e-instruct",
      service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
      compartment_id=OCI_COMPARTMENT_ID,
      )
  result = llm.invoke(prompt_with_image)
  print(f"Result: {result}") 
  return result.content

In [66]:
@tool
def get_text_with_image(query: str, content: Document) -> str:
    """
    Get Revenue or net income for the past years by image.
    Args:
        query (str): The query to ask the model.
    Returns:
        str: The answer from the model.
    """

    id = content.metadata['id']
    file_name = os.path.basename(content.metadata['file_path'])
    print(f"file_name: {file_name}")
    try:
        with oracledb.connect(user=UN, password=PW, dsn=DSN) as connection:
            with connection.cursor() as cursor:
                cursor.setinputsizes(oracledb.DB_TYPE_VECTOR)
                select_sql = f"""
                    SELECT
                        image_blob
                    FROM
                        embedding_contents
                    WHERE id = :1
                """
                cursor.execute(select_sql, [id])
                blob, = cursor.fetchone()
                offset = 1
                bufsize = 65536

                with open("../images/" + file_name, 'wb') as f:
                    while True:
                        data = blob.read(offset, bufsize)
                        if data:
                            f.write(data)
                        if len(data) < bufsize:
                            break
                        offset += bufsize
            connection.close()
                    
    except oracledb.DatabaseError as e:
        print(f"Database error: {e}")
        raise
    except Exception as e:
        print("Error Vector Search:", e)
    
    
    
    prompt = ChatPromptTemplate([
        ("system", "あなたは言語翻訳のAIアシスタントです。日本語を英語に翻訳してください。"),
        ("human", "{input} "),
    ])
    
    llm = ChatOCIGenAI(
        model_id="cohere.command-a-03-2025",
        service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
        compartment_id=OCI_COMPARTMENT_ID,
    )
    chain = {'input': RunnablePassthrough()} | prompt | llm | StrOutputParser()
    query_en = chain.invoke({"input":query})

    # result_images = get_text_by_image(query)
    # print(result_images)
    image_path = "../images/" + os.path.basename(content.metadata['file_path'])
    res = chat_with_image(
        image_path=image_path,
        prompt=query_en,
        system_prompt=(
            "You are a AI assistant. Please answer the question based on the image.\n"

            "** Question **"
            f"{query}"

            "** Output **\n"
            "- FYXX: 1234\n"
            "- FYXX: 5678\n"
        )
    )
    return res

### Agent

In [67]:
agent = create_react_agent(
    model=model,
    tools=[get_content, get_text_with_markdown, get_text_with_image],
    prompt=(
        "You are a chat agent. You can call three tools:\n"
        "- a get_content. Assign getting content from the database to this tool\n"
        "- a get_text_with_markdown. Assign getting department expenses, sales, and operating income to this tool\n"
        "- a get_text_with_image. Assign getting revenue or net income for the past years tasks to this tool\n"

        "You must call the tools in following order.\n"
        "1. get_content\n"
        "You pass the query as an argument to the tool.\n"
        "2. get_text_with_markdown or get_text_with_image\n"
        "You pass the query and the content from the first tool as arguments to the tool.\n"
        "If the response from get_content is an image, you must call get_text_with_image.\n"
        "If the response from get_content is text, you must call get_text_with_markdown.\n"

        "Do not do any work yourself."
        "If you need to use a tool, please use the tool and return the result.\n"
        "You must answer in Japanese.\n"
    ),
    debug=True,
)

In [68]:
result_markdown = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "2024/5のQ4のFacilityの経費を教えてください。",
        },
    ]
})

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': '2024/5のQ4のFacilityの経費を教えてください。', 'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': '2024/5のQ4のFacilityの経費を教えてください。', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='2024/5のQ4のFacilityの経費を教えてください。', additional_kwargs={}, response_metadata={}, id='f7d27249-b1a0-4fb4-9f7b-6b3787fbe41f')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='2024/5のQ4のFacilityの経費を教えてください。', additional_kwargs={}, response_metadata={}, id='f7d27249-b1a0-4fb4-9f7b-6b3787fbe41f')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='まず、get_contentツールを使用してデータベースからコンテンツを取得し、次にget_text_with_markdownツールを使用して2024年5月の第4四半期のFacilityの経費を取得します。', additional_kwargs={'docume

In [69]:
print(result_markdown)

{'messages': [HumanMessage(content='2024/5のQ4のFacilityの経費を教えてください。', additional_kwargs={}, response_metadata={}, id='f7d27249-b1a0-4fb4-9f7b-6b3787fbe41f'), AIMessage(content='まず、get_contentツールを使用してデータベースからコンテンツを取得し、次にget_text_with_markdownツールを使用して2024年5月の第4四半期のFacilityの経費を取得します。', additional_kwargs={'documents': None, 'citations': None, 'search_queries': None, 'is_search_required': None, 'finish_reason': 'COMPLETE', 'tool_calls': [{'id': '5d642087761144129dea43c21610fcea', 'function': {'name': 'get_content', 'arguments': '{"query": "2024\\u5e745\\u6708\\u306e\\u7b2c4\\u56db\\u534a\\u671f\\u306eFacility\\u306e\\u7d4c\\u8cbb"}'}, 'type': 'function'}]}, response_metadata={'model_id': 'cohere.command-a-03-2025', 'model_version': '1.0', 'request_id': 'FF2A63B1F9A249DCB605BA7D7821B81C/DA3C5ECAF597E7B1E711E08FBA4A6C4F/5B9A56336B7F0CDE3DD148C36747819B', 'content-length': '820', 'documents': None, 'citations': None, 'search_queries': None, 'is_search_required': None, 'finish_reason': 'COMPLETE

In [70]:
result_image = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "5年間の売り上げの推移を教えてください。",
        },
    ]
})

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': '5年間の売り上げの推移を教えてください。', 'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': '5年間の売り上げの推移を教えてください。', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='5年間の売り上げの推移を教えてください。', additional_kwargs={}, response_metadata={}, id='a9e3ccef-5fd3-4308-b166-9c299cb3758f')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='5年間の売り上げの推移を教えてください。', additional_kwargs={}, response_metadata={}, id='a9e3ccef-5fd3-4308-b166-9c299cb3758f')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='まず、get_contentツールを使用して、5年間の売上高の推移に関する情報を取得します。次に、結果がテキストか画像かによって、get_text_with_markdownまたはget_text_with_imageツールを使用します。', additional_kwargs={'documents': None, 'citations': 

In [71]:
print(result_image)

{'messages': [HumanMessage(content='5年間の売り上げの推移を教えてください。', additional_kwargs={}, response_metadata={}, id='a9e3ccef-5fd3-4308-b166-9c299cb3758f'), AIMessage(content='まず、get_contentツールを使用して、5年間の売上高の推移に関する情報を取得します。次に、結果がテキストか画像かによって、get_text_with_markdownまたはget_text_with_imageツールを使用します。', additional_kwargs={'documents': None, 'citations': None, 'search_queries': None, 'is_search_required': None, 'finish_reason': 'COMPLETE', 'tool_calls': [{'id': '278b5ef464b94d52be0ac4eb7972ad7a', 'function': {'name': 'get_content', 'arguments': '{"query": "5\\u5e74\\u9593\\u306e\\u58f2\\u308a\\u4e0a\\u3052\\u306e\\u63a8\\u79fb"}'}, 'type': 'function'}]}, response_metadata={'model_id': 'cohere.command-a-03-2025', 'model_version': '1.0', 'request_id': '8AA9F784AE054A2BA597FD319E1B8A11/C29180A5D8C1D8FED46F3394B8F091C4/E4B916410229537D8CE22D5240477696', 'content-length': '845', 'documents': None, 'citations': None, 'search_queries': None, 'is_search_required': None, 'finish_reason': 'COMPLETE', 'tool_calls'